In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
import pylab
import requests 
import pprint
import time
import csv
MINUTES_BETWEEN_READS = 0.1 #limit = 10 API calls per minute, 500 per day
past_rain = [];
rain_day = []; 
past_temp = [];
temp_day = [];
for i in range(2003,2017):
    #wunderground only had historical weather reports of Portugal back to 2003 and the Wine enthusiast 
    for j in (4,10):
        #gathering temperatures from the months of April to September 
        for k in range(1,10):
            #single digit days of the month to get the right format for the API call, also allows us to use two different API keys
            past_response = requests.get('http://api.wunderground.com/api/b7b6901fb869c948/history_' + str(i) + '0' + str(j) + '0' + str(k) + '/geolookup/conditions/q/Portugal/Lisbon.json').json()
            if past_response['history']['dailysummary']:
                temp_day.append(float(past_response['history']['dailysummary'][0]['meantempi']))
                #if specific day has no record, skip that date
            time.sleep(60*MINUTES_BETWEEN_READS)
        for k in range(10,31):
            past_response = requests.get('http://api.wunderground.com/api/252679f1146ecf24/history_' + str(i) + '0' + str(j) + str(k) + '/geolookup/conditions/q/Portugal/Lisbon.json').json()
            if past_response['history']['dailysummary']:
                temp_day.append(float(past_response['history']['dailysummary'][0]['meantempi']))
            time.sleep(60*MINUTES_BETWEEN_READS)
    past_temp.append(sum(temp_day)/float(len(temp_day))) #averages the temperature from everyday from April to September
    for j in range(8,10):
        #gathering total rainfall from August and September 
        for k in range(1,10):
            past_response = requests.get('http://api.wunderground.com/api/b7b6901fb869c948/history_' + str(i) + '0' + str(j) + '0' + str(k) + '/geolookup/conditions/q/Portugal/Lisbon.json').json()
            if past_response['history']['dailysummary']:
                rain_day.append(float(past_response['history']['dailysummary'][0]['rain']))
            time.sleep(60*MINUTES_BETWEEN_READS)
        for k in range(10,31):
            past_response = requests.get('http://api.wunderground.com/api/252679f1146ecf24/history_' + str(i) + '0' + str(j) + str(k) + '/geolookup/conditions/q/Portugal/Lisbon.json').json()
            if past_response['history']['dailysummary']:
                rain_day.append(float(past_response['history']['dailysummary'][0]['rain']))
            time.sleep(60*MINUTES_BETWEEN_READS)
    past_rain.append(sum(rain_day)) #adds the total rainfall for each day 

print(past_rain) #total rainfall for years 2003-2016 in months August and Spetember, array with element for each year
print(past_temps) #average temper


In [7]:
import pandas as pd
from sklearn.cross_validation import train_test_split


df = pd.read_csv('weatherANDwine.csv')
# this csv should be made when calling the wunderground API, but due to the call limit, the data needs to be 
# entered in manually
Temp_and_Rain = df[list(df.columns)[3:]]
# variables
Ports_Quality = df['Ports']
# 1st dependent variable
TableWine_Quality = df['Table Wines (Red)']
# 2nd dependent variable
Temp_and_Rain_train, Temp_and_Rain_test, Ports_Quality_train, Ports_Quality_test = train_test_split(Temp_and_Rain, Ports_Quality)
# training data (80%/20%)
Temp_and_Rain_train, Temp_and_Rain_test, TableWine_Quality_train, TableWine_Quality_test = train_test_split(Temp_and_Rain, TableWine_Quality)
# training data (80%/20%)






In [8]:
import statsmodels.api as sm
ols_model = sm.OLS(Ports_Quality, Temp_and_Rain, missing='drop')
ols_results = ols_model.fit()
print(ols_results.summary())
# linear regressionn for 1st dependent variable (Port Quality)

                            OLS Regression Results                            
Dep. Variable:                  Ports   R-squared:                       0.998
Model:                            OLS   Adj. R-squared:                  0.997
Method:                 Least Squares   F-statistic:                     2487.
Date:                Wed, 02 May 2018   Prob (F-statistic):           1.94e-16
Time:                        16:16:40   Log-Likelihood:                -41.062
No. Observations:                  14   AIC:                             86.12
Df Residuals:                      12   BIC:                             87.40
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
temp           1.3465      0.039     34.516      0.0

/Users/isabelaosorio/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:1334: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=14
  "anyway, n=%i" % int(n))


In [9]:
ols_model = sm.OLS(TableWine_Quality, Temp_and_Rain, missing='drop')
ols_results = ols_model.fit()
print(ols_results.summary())
# linear regressionn for 2nd dependent variable (Table Wine (Reds) Quality)

                            OLS Regression Results                            
Dep. Variable:      Table Wines (Red)   R-squared:                       0.998
Model:                            OLS   Adj. R-squared:                  0.997
Method:                 Least Squares   F-statistic:                     2714.
Date:                Wed, 02 May 2018   Prob (F-statistic):           1.15e-16
Time:                        16:16:42   Log-Likelihood:                -40.392
No. Observations:                  14   AIC:                             84.78
Df Residuals:                      12   BIC:                             86.06
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
temp           1.3274      0.037     35.691      0.0

/Users/isabelaosorio/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:1334: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=14
  "anyway, n=%i" % int(n))
